In [ ]:
exit

In [1]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.utils.data as D
import torchvision.utils as vutils
from torchvision import datasets
from torch.autograd import Variable
from torch.nn.parameter import Parameter
from torch.utils.data import Dataset, DataLoader

from torchsummary import summary

import torchvision
import torchvision.transforms as transforms
from torchvision import datasets
import torchvision.models as models
import os
import copy
import time
import csv

from tensorboardX import SummaryWriter

import matplotlib.pyplot as plt

In [2]:
# import sys
# sys.path.insert(0,'/home/lee/Research/128size_modelpy/')
# from ResNetaddone224 import ResNet18
# # from ResNet import ResNet18

In [3]:
# Training settings
batch_size = 64

In [8]:
#%%
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(224),
        transforms.RandomHorizontalFlip(),
        transforms.ColorJitter(),
        transforms.RandomCrop(224),
        transforms.RandomResizedCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
#         transforms.Scale(256),
#         transforms.CenterCrop(224),
        transforms.Resize(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}
    
data_dir = '/home/lee/Research/10.jinsu/jaffe'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'test', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'test', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'test', 'val']}
print(dataset_sizes['train'], dataset_sizes['test'], dataset_sizes['val'])
class_names = image_datasets['train'].classes
print(class_names)

RuntimeError: Found 0 files in subfolders of: /home/lee/Research/10.jinsu/jaffe/train
Supported extensions are: .jpg,.jpeg,.png,.ppm,.bmp,.pgm,.tif

In [ ]:
# print(pre_trained_model1)
resnet_structure = models.resnet18(pretrained=True)
num_ftrs = resnet_structure.fc.in_features
resnet_structure.fc = nn.Linear(num_ftrs, num_ftrs)

In [ ]:
lt=8
cntr=0

for child in pre_trained_model.children():
    cntr+=1

    if cntr < lt:
        # print child
        for param in child.parameters():
            param.requires_grad = False

In [ ]:
lt=8
cntr=0

for child in net_Image.children():
    cntr+=1

    if cntr < lt:
        # print child
        for param in child.parameters():
            param.requires_grad = False

In [ ]:
class DeepNeuralDecisionForest_res_ImgNet(nn.Module):
    def __init__(self, p_keep_conv, p_keep_hidden, n_leaf, n_label, n_tree, n_depth):
        super(DeepNeuralDecisionForest_res_ImgNet, self).__init__()
        self.pre_train = pre_trained_model
        self.net_ImageNet = net_Image
#         self.fc = nn.Linear(1024,512)
        self._nleaf = n_leaf
        self._nlabel = n_label
        self._ntree = n_tree
        self._ndepth = n_depth
        self._batchsize = batch_size
    
        self.treelayers = nn.ModuleList()
        self.pi_e = nn.ParameterList()
        for i in range(self._ntree):
            treelayer = nn.Sequential()
            treelayer.add_module('sub_linear1', nn.Linear(1024, 128))
            treelayer.add_module('sub_relu', nn.ReLU())
            treelayer.add_module('sub_drop1', nn.Dropout(1-p_keep_hidden))
            treelayer.add_module('sub_linear2', nn.Linear(128, self._nleaf))
            treelayer.add_module('sub_sigmoid', nn.Sigmoid())
           
            self.treelayers.append(treelayer)
            self.pi_e.append(Parameter(self.init_prob_weights([self._nleaf, self._nlabel], -2, 2)))

    def init_pi(self):
        return torch.ones(self._nleaf, self._nlabel)/float(self._nlabel)

    def init_weights(self, shape):
        return torch.randn(shape).uniform(-0.01,0.01)

    def init_prob_weights(self, shape, minval=-5, maxval=5):
        return torch.Tensor(shape[0], shape[1]).uniform_(minval, maxval)

    def compute_mu(self, flat_decision_p_e):
        n_batch = self._batchsize
        batch_0_indices = torch.range(0, n_batch * self._nleaf - 1, self._nleaf).unsqueeze(1).repeat(1, self._nleaf).long()#64x16
        
#        print('batch_0_indices', batch_0_indices,'shape',batch_0_indices.shape)
        
        in_repeat = self._nleaf // 2
        out_repeat = n_batch
        
        batch_complement_indices = torch.LongTensor(
            np.array([[0] * in_repeat, [n_batch * self._nleaf] * in_repeat] * out_repeat).reshape(n_batch, self._nleaf))#128x8->64x16
#        print('batch_complement_indices', bamodeltch_complement_indices, 'shape', batch_complement_indices.shape)

        
        # First define the routing probabilistics d for root nodes
        mu_e = []
        indices_var = Variable((batch_0_indices + batch_complement_indices).view(-1)) #1024
        indices_var = indices_var.cuda()
        # iterate over each tree
        for i, flat_decision_p in enumerate(flat_decision_p_e):
#            print('f',flat_decision_p, len(flat_decision_p))
            mu = torch.gather(flat_decision_p, 0, indices_var).view(n_batch, self._nleaf)

            mu_e.append(mu)#10-> number of tree

        # from the second layer to the last layer, we make the decison nodes
        for d in range(1, self._ndepth + 1):
            indices = torch.range(2 ** d, 2 ** (d + 1) - 1) - 1
#            print('indices',indices)
            tile_indices = indices.unsqueeze(1).repeat(1, 2 ** (self._ndepth - d + 1)).view(1, -1)
#            print('tile', tile_indices)
            batch_indices = batch_0_indices + tile_indices.repeat(n_batch, 1).long()
#            print('batch', batch_indices)
            in_repeat = in_repeat // 2
            out_repeat = out_repeat * 2
            # Again define the indices that picks d and 1-d for the nodes
            batch_complement_indices = torch.LongTensor(
                np.array([[0] * in_repeat, [n_batch * self._nleaf] * in_repeat] * out_repeat).reshape(n_batch, self._nleaf))
#            print(batch_complement_indices)
            mu_e_update = []
            indices_var = Variable((batch_indices + batch_complement_indices).view(-1))
#            print(indices_var)
            indices_var = indices_var.cuda()
            for mu, flat_decision_p in zip(mu_e, flat_decision_p_e):
#                xxx = torch.mul(mu, torch.gather(flat_decision_p, 0, indices_var)
#                print(xxx)
                mu = torch.mul(mu, torch.gather(flat_decision_p, 0, indices_var).view(
                    n_batch, self._nleaf))

                mu_e_update.append(mu)
            mu_e = mu_e_update
        return mu_e
#        print(mu_e)
    def compute_py_x(self, mu_e, leaf_p_e):
        py_x_e = []
        n_batch = self._batchsize

        for i in range(len(mu_e)):
            py_x_tree = mu_e[i].unsqueeze(2).repeat(1, 1, self._nlabel).mul(leaf_p_e[i].unsqueeze(0).repeat(n_batch, 1, 1)).mean(1)
            py_x_e.append(py_x_tree.squeeze().unsqueeze(0))
        py_x_e = torch.cat(py_x_e, 0)
        py_x = py_x_e.mean(0).squeeze()
        return py_x

    def forward(self, x):
        feat = self.pre_train.forward(x)
        feat_imagenet = self.net_ImageNet.forward(x)
        feat_combine = torch.cat((feat,feat_imagenet),dim=1)
#         feat_combine = self.fc(feat_combine)
        self._batchsize = x.size(0)

        flat_decision_p_e = []
        leaf_p_e = []
#        print(len(self.treelayers))#50, number of tree
        for i in range(len(self.treelayers)):
            decision_p = self.treelayers[i].forward(feat_combine)#64x16

            decision_p_comp = 1 - decision_p

            decision_p_pack = torch.cat((decision_p, decision_p_comp))
            flat_decision_p = decision_p_pack.view(-1)
            
            flat_decision_p_e.append(flat_decision_p)
        
            leaf_p = F.softmax(self.pi_e[i])
#            print(leaf_p)
            leaf_p_e.append(leaf_p)
#        print(flat_decision_p_e) 
        mu_e = self.compute_mu(flat_decision_p_e)
#        print('mu_e',mu_e)
        py_x = self.compute_py_x(mu_e, leaf_p_e)#py_x length: batch_size, py_x.shape: batch_size x len(mu_e) 
        
#        print(py_x)
        return torch.log(py_x)


In [ ]:
################ Definition ######################### 
DEPTH = 3  # Depth of a tree
N_LEAF = 2 ** (DEPTH + 1)  # Number of leaf node
N_LABEL = len(class_names)  # Number of classes
N_TREE = 5 # Number of trees (ensemble)
N_BATCH = batch_size#args.batch-size  # Number of data points per mini-batch
# network hyperparameters
p_conv_keep = 0.5
p_full_keep = 0.5

In [ ]:
 # load the pre-trained weights
model_file = '20181127resnet18+ImageNet224pre-data_9-loss-Adam(0.0001)-epoch(100)-class(9).pt'
path = '/home/lee/Research/pytorch-tree-building/Trained/resnet18/' + model_file
model1 = DeepNeuralDecisionForest_res_ImgNet(p_keep_conv = p_conv_keep, p_keep_hidden 
                                  = p_full_keep, n_leaf= N_LEAF, n_label= N_LABEL, n_tree= N_TREE, n_depth= DEPTH)
model1.load_state_dict(torch.load(path))
# print(model1)

In [ ]:
# pre_resnet = models.resnet18(pretrained=False)

In [ ]:
 # load the pre-trained weights
model_file = '20181127only-pre-resnet18-data_9-loss-Adam(0.0001)-epoch(100)-class(9).pt'
path = '/home/lee/Research/pytorch-tree-building/Trained/resnet18/' + model_file
model2 = pre_trained_model
model2.fc = nn.Linear(512,15)
# print(model2)
model2.load_state_dict(torch.load(path))
model2.fc = nn.Linear(512,512)
# print(model2)


In [ ]:
model1 = nn.Sequential(*list(model1.children())[:-2])
model1.add_module('fc',nn.Linear(512,512))

print(model1)

In [ ]:
model1.fc

In [ ]:
##model combine
class DeepNeuralDecisionForest_combine(nn.Module):
    def __init__(self, p_keep_conv, p_keep_hidden, n_leaf, n_label, n_tree, n_depth):
        super(DeepNeuralDecisionForest_combine, self).__init__()
        self.pre_train = model1
        self.pre_train_resnet = model2
#         self.pre_train2 = pre_trained_model2
#         self.net_ImageNet = net
        
        self._nleaf = n_leaf
        self._nlabel = n_label
        self._ntree = n_tree
        self._ndepth = n_depth
        self._batchsize = batch_size

    
        self.treelayers = nn.ModuleList()
        self.pi_e = nn.ParameterList()
        for i in range(self._ntree):
            treelayer = nn.Sequential()
            treelayer.add_module('sub_linear1', nn.Linear(1024, 64))
            treelayer.add_module('sub_relu', nn.ReLU())
            treelayer.add_module('sub_drop1', nn.Dropout(1-p_keep_hidden))
            treelayer.add_module('sub_linear2', nn.Linear(64, self._nleaf))
            treelayer.add_module('sub_sigmoid', nn.Sigmoid())
           
            self.treelayers.append(treelayer)
            self.pi_e.append(Parameter(self.init_prob_weights([self._nleaf, self._nlabel], -2, 2)))

    def init_pi(self):
        return torch.ones(self._nleaf, self._nlabel)/float(self._nlabel)

    def init_weights(self, shape):
        return torch.randn(shape).uniform(-0.01,0.01)

    def init_prob_weights(self, shape, minval=-5, maxval=5):
        return torch.Tensor(shape[0], shape[1]).uniform_(minval, maxval)

    def compute_mu(self, flat_decision_p_e):
        n_batch = self._batchsize
        batch_0_indices = torch.range(0, n_batch * self._nleaf - 1, self._nleaf).unsqueeze(1).repeat(1, self._nleaf).long()#64x16
        
#        print('batch_0_indices', batch_0_indices,'shape',batch_0_indices.shape)
        
        in_repeat = self._nleaf // 2
        out_repeat = n_batch
        
        batch_complement_indices = torch.LongTensor(
            np.array([[0] * in_repeat, [n_batch * self._nleaf] * in_repeat] * out_repeat).reshape(n_batch, self._nleaf))#128x8->64x16
#        print('batch_complement_indices', bamodeltch_complement_indices, 'shape', batch_complement_indices.shape)

        
        # First define the routing probabilistics d for root nodes
        mu_e = []
        indices_var = Variable((batch_0_indices + batch_complement_indices).view(-1)) #1024
        indices_var = indices_var.cuda()
        # iterate over each tree
        for i, flat_decision_p in enumerate(flat_decision_p_e):
#            print('f',flat_decision_p, len(flat_decision_p))
            mu = torch.gather(flat_decision_p, 0, indices_var).view(n_batch, self._nleaf)

            mu_e.append(mu)#10-> number of tree

        # from the second layer to the last layer, we make the decison nodes
        for d in range(1, self._ndepth + 1):
            indices = torch.range(2 ** d, 2 ** (d + 1) - 1) - 1
#            print('indices',indices)
            tile_indices = indices.unsqueeze(1).repeat(1, 2 ** (self._ndepth - d + 1)).view(1, -1)
#            print('tile', tile_indices)
            batch_indices = batch_0_indices + tile_indices.repeat(n_batch, 1).long()
#            print('batch', batch_indices)
            in_repeat = in_repeat // 2
            out_repeat = out_repeat * 2
            # Again define the indices that picks d and 1-d for the nodes
            batch_complement_indices = torch.LongTensor(
                np.array([[0] * in_repeat, [n_batch * self._nleaf] * in_repeat] * out_repeat).reshape(n_batch, self._nleaf))
#            print(batch_complement_indices)
            mu_e_update = []
            indices_var = Variable((batch_indices + batch_complement_indices).view(-1))
#            print(indices_var)
            indices_var = indices_var.cuda()
            for mu, flat_decision_p in zip(mu_e, flat_decision_p_e):
#                xxx = torch.mul(mu, torch.gather(flat_decision_p, 0, indices_var)
#                print(xxx)
                mu = torch.mul(mu, torch.gather(flat_decision_p, 0, indices_var).view(
                    n_batch, self._nleaf))

                mu_e_update.append(mu)
            mu_e = mu_e_update
        return mu_e
#        print(mu_e)
    def compute_py_x(self, mu_e, leaf_p_e):
        py_x_e = []
        n_batch = self._batchsize

        for i in range(len(mu_e)):
            py_x_tree = mu_e[i].unsqueeze(2).repeat(1, 1, self._nlabel).mul(leaf_p_e[i].unsqueeze(0).repeat(n_batch, 1, 1)).mean(1)
            py_x_e.append(py_x_tree.squeeze().unsqueeze(0))
        py_x_e = torch.cat(py_x_e, 0)
        py_x = py_x_e.mean(0).squeeze()
        return py_x

    def forward(self, x):
        feat1 = self.pre_train.forward(x)
        feat2 = self.pre_train_resnet.forward(x)

        feat_combine = torch.cat((feat1,feat2),dim=1)
        feat = nn.Linear(feat_combine, 1024)
        self._batchsize = x.size(0)

        flat_decision_p_e = []
        leaf_p_e = []
#        print(len(self.treelayers))#50, number of tree
        for i in range(len(self.treelayers)):
            decision_p = self.treelayers[i].forward(feat)#64x16

            decision_p_comp = 1 - decision_p

            decision_p_pack = torch.cat((decision_p, decision_p_comp))
            flat_decision_p = decision_p_pack.view(-1)
            
            flat_decision_p_e.append(flat_decision_p)
        
            leaf_p = F.softmax(self.pi_e[i])
#            print(leaf_p)
            leaf_p_e.append(leaf_p)
#        print(flat_decision_p_e) 
        mu_e = self.compute_mu(flat_decision_p_e)
#        print('mu_e',mu_e)
        py_x = self.compute_py_x(mu_e, leaf_p_e)#py_x length: batch_size, py_x.shape: batch_size x len(mu_e) 
        
#        print(py_x)
        return torch.log(py_x)


In [ ]:
#first training the model
model_combine = DeepNeuralDecisionForest_combine(p_keep_conv = p_conv_keep, p_keep_hidden 
                                 = p_full_keep, n_leaf= N_LEAF, n_label= N_LABEL, n_tree= N_TREE, n_depth= DEPTH)
use_gpu = torch.cuda.is_available()
if use_gpu:
    print("Using CUDA, model")
    model_combine.cuda() #after second other epoch model

summary(model_combine,(3,224,224))

In [ ]:
print(model_combine)

In [ ]:
writer = SummaryWriter()

In [ ]:
def train_model(model, criterion, optimizer, optimizer_name, learning_rate, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    best_epoch = 0
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch + 1, num_epochs))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
#                 scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.cuda()
                labels = labels.cuda()

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    dummy_img = outputs
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
        
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            if phase == 'train':
                writer.add_scalar('data/train_loss_combine_data_9', epoch_loss, epoch)
                writer.add_scalar('data/train_acc_combine_data_9', epoch_acc, epoch)
            else:
                writer.add_scalar('data/val_loss_combine_data_9', epoch_loss, epoch)
                writer.add_scalar('data/val_acc_combine_data_9', epoch_acc, epoch)
            
#             x[epoch] = vutils.make_grid(dummy_img, normalize=True, scale_each=True)
#             writer.add_image('Image', x[epoch], epoch)
            for name, param in model.named_parameters():
                writer.add_histogram(name, param.clone().cpu().data.numpy(), epoch)
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))
            csvfile = open(os.path.join('./csv/resnet18/', '20181128DecisionForest_resnet18_224_combine_data_9_places_{}{}_class{}_epoch{}_2.csv'.format(optimizer_name, learning_rate, len(class_names), num_epochs)), 'a', newline='')
            csv_writer = csv.writer(csvfile, delimiter=';', quotechar='|', quoting=csv.QUOTE_MINIMAL)
            csv_writer.writerow(['class', len(class_names), 'epoch', epoch, phase, epoch_loss, epoch_acc])
            csvfile.close()
            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_epoch = epoch
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f} at {}'.format(best_acc,best_epoch+1))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model


In [ ]:
 #second other model2 training
model2 = DeepNeuralDecisionForest(p_keep_conv = p_conv_keep, p_keep_hidden 
                                  = p_full_keep, n_leaf= N_LEAF, n_label= N_LABEL, n_tree= N_TREE, n_depth= DEPTH)

use_gpu = torch.cuda.is_available()
if use_gpu:
    print("Using CUDA, model2")
    model2.cuda() #after second other epoch model


In [ ]:
 # load the pre-trained weights
model_file = '20181123resnet18224pre-transforms-loss-Adam(0.0001)-epoch(100)-class(15).pt'
path = '/home/lee/Research/pytorch-tree-building/Trained/resnet18/' + model_file

model2.load_state_dict(torch.load(path))
#
# summary(model2, (3,128,128))

In [ ]:
num_epoch = 100
criterion = nn.CrossEntropyLoss()
optimizer_name = 'Adam'
learning_rate = 0.0001
momentum = 0.9
optimizer = optim.Adam(model_combine.parameters(), lr=learning_rate)
# Decay LR by a factor of 0.1 every 7 epochs
# exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.1)
#for epoch in range(1, num_epoch+1):
model_final = train_model(model_combine, criterion, optimizer, optimizer_name, learning_rate, num_epoch)
torch.save(model_final.state_dict(), './Trained/resnet18/20181128resnet18-combine-data_9-loss-{}({})-epoch({})-class({}).pt'.format(optimizer_name, 
           learning_rate, num_epoch, len(class_names)))

In [ ]:
exit

In [ ]:
################# Test model #################

In [ ]:
 # load the pre-trained weights
model_file = '20181127resnet18-data_9-loss-Adam(0.0001)-epoch(100)-class(9).pt'
path = '/home/lee/Research/pytorch-tree-building/Trained/resnet18/' + model_file
model3 = DeepNeuralDecisionForest(p_keep_conv = p_conv_keep, p_keep_hidden 
                                  = p_full_keep, n_leaf= N_LEAF, n_label= N_LABEL, n_tree= N_TREE, n_depth= DEPTH)
model3.load_state_dict(torch.load(path))
use_gpu = torch.cuda.is_available()
if use_gpu:
    print("Using CUDA, model3")
    model3.cuda() #after second other epoch model

In [ ]:
def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].view(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

In [ ]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [ ]:
criterion = nn.CrossEntropyLoss()
def validate(model, criterion):
    

    # switch to evaluate mode
    model.eval()

    with torch.no_grad():
        end = time.time()
        for phase in ['test','val']:
            print(phase)
            batch_time = AverageMeter()
            losses = AverageMeter()
            top1 = AverageMeter()
            top5 = AverageMeter()
            for i, (inputs, labels) in enumerate(dataloaders[phase]):
                if model.cuda:
                    inputs = inputs.cuda()
                    labels = labels.cuda()
                inputs, labels = Variable(inputs, volatile=True), Variable(labels)

                # compute output
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                # measure accuracy and record loss
                acc1, acc5 = accuracy(outputs, labels, topk=(1, 5))
                losses.update(loss.item(), inputs.size(0))
                top1.update(acc1[0], inputs.size(0))
                top5.update(acc5[0], inputs.size(0))

                # measure elapsed time
                batch_time.update(time.time() - end)
                end = time.time()

                if i % 10 == 0:
                    print('Test: [{0}/{1}]\t'
                          'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                          'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                          'Acc@1 {top1.val:.3f} ({top1.avg:.3f})\t'
                          'Acc@5 {top5.val:.3f} ({top5.avg:.3f})'.format(
                           i, len(dataloaders[phase]), batch_time=batch_time, loss=losses,
                           top1=top1, top5=top5))
            
            print(' * Acc@1 {top1.avg:.3f} Acc@5 {top5.avg:.3f}'
                  .format(top1=top1, top5=top5))
            

    return top1.avg

In [ ]:
#############check top1 and top5 acc#############
validate(model3,criterion)

In [ ]:
def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure(figsize=(20,20))

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['test']):
            inputs = inputs.cuda()
            labels = labels.cuda()
            
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            print(outputs)
            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
#                 ax = plt.figure()
                ax.axis('off')
                ax.set_title('predicted: {}, labels: {}'.format(class_names[preds[j]],class_names[labels[j]]))
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return model.train(mode=was_training)

In [ ]:
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(1)  # pause a bit so that plots are updated
    

In [ ]:


def visualize_model(model, num_images=6):
    model.eval()
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['test']):
            
            inputs = inputs.cuda()
            labels = labels.cuda()

            outputs = model(inputs)
            
            _, preds = torch.max(outputs, 1)
#             print(outputs)
            sm = torch.nn.Softmax()
            outputs = sm(outputs)

            top5_prob, top5_label = torch.topk(outputs,5)
#             print(inputs.size(0))#inputs.size(0): batch_size, inputs.size(1): channel, inputs.size(2,3):width, height
#             for k in range(5):
#                 print('{}: {}%'.format(class_names[top5_label[1][k]],top5_prob[1][k]*100))
#             print(top5_prob)
            for j in range(num_images):
                for k in range(5):
                    print('{}: {:.4f}%'.format(class_names[top5_label[j][k]],top5_prob[j][k]*100))

                print('predicted: {}, labels: {}'.format(class_names[preds[j]],class_names[labels[j]]))
                plt.axis('off')
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title('predicted: {}, labels: {}'.format(class_names[preds[j]],class_names[labels[j]]))
                
                
                imshow(inputs.cpu().data[j])

In [ ]:
visualize_model(model3,6)

In [ ]:
exit